In [ ]:
import sys, os
from google.colab import drive
import pandas as pd
drive.mount('/content/drive', force_remount=True)

sys.path.append('/content/drive/My Drive/radiomics/Radiomics Workshop')
os.chdir('drive/My Drive/radiomics/Radiomics Workshop')

!pip install SimpleITK==1.2.4
!pip install pyradiomics===3.0

Mounted at /content/drive
     |████████████████████████████████| 42.5MB 99kB/s 
  Found existing installation: SimpleITK 2.0.0
    Uninstalling SimpleITK-2.0.0:
      Successfully uninstalled SimpleITK-2.0.0


In [ ]:
import torch
import SimpleITK as sitk
from utils import tensors_as_images
from PIL import Image
import numpy as np
from os import listdir
from os.path import isfile, join
import cv2
from create_data_set import downsamplePatient

def load(mouse, slice_num, folder="datasets/newMiceData", is_png=True, to_resize=False):
  if is_png:
    image_path = folder + "/" + mouse + "/images/Slice" + str(f"{slice_num:04d}.png")
    img = sitk.GetArrayFromImage(sitk.ReadImage(image_path))
  else:
    folder_path = folder + "/" + mouse + "/images"
    img_names = [f for f in listdir(folder_path) if isfile(join(folder_path, f))]
    img = sitk.ReadImage(folder_path+"/"+img_names[slice_num-1])
    img = sitk.IntensityWindowing(img, -1000, 1000, 0, 255)
    img = sitk.Cast(img, sitk.sitkUInt8)
    if to_resize:
      img = downsamplePatient(img, reduced_dim=True)
      img = sitk.GetArrayFromImage(img)
    else:
      img = sitk.GetArrayFromImage(img)[0]

  color_img = cv2.cvtColor(img ,cv2.COLOR_GRAY2RGB)
  return color_img


def mark_patch(slice_img, patch_tuple, colour):
  row = patch_tuple[0]
  col = patch_tuple[1]
  if colour=="red": #cluster 0
    c1, c2, c3 = 255, 0, 0
  elif colour=="blue": #cluster 1
    c1, c2, c3 = 0, 0, 255
  elif colour=="yellow": #cluster 2
    c1, c2, c3 = 255, 255, 0
  else:
    c1, c2, c3 = 0, 255, 0

  for i in range(col*50, (col+1)*50):
    slice_img[row*50][i][0] = c1
    slice_img[row*50][i][1] = c2
    slice_img[row*50][i][2] = c3
    slice_img[(row+1)*50-1][i][0] = c1
    slice_img[(row+1)*50-1][i][1] = c2
    slice_img[(row+1)*50-1][i][2] = c3

  for i in range(row*50, (row+1)*50):
    slice_img[i][col*50][0] = c1
    slice_img[i][col*50][1] = c2
    slice_img[i][col*50][2] = c3
    slice_img[i][(col+1)*50-1][0] = c1
    slice_img[i][(col+1)*50-1][1] = c2
    slice_img[i][(col+1)*50-1][2] = c3
 

In [ ]:
#df.groupby(['Name', "Slices",'Clusters']).count()
#df.groupby('Name').count()
#df.loc[(df['Name'] == "MB1317200714F000H00000000C000000") & (df['Slices'] == "(199, 199)")]

In [ ]:
d = {0:"red", 1:"blue", -1:"green", 2: "yellow"}

def get_data(file_name):
    df = pd.read_csv(file_name, sep=',')
    return df

def mark_patches(data, mouse_name, folder_data="datasets/newMiceData", network="segmentation", is_png=True, to_resize=False):
  mouse_data = data.loc[data['Name'] == mouse_name]
  slices_list = list(dict.fromkeys(mouse_data['Slices'].tolist()))
  path = "results/patches_mark/"+ mouse_name + "/" + network
  if not os.path.isdir(path):
    os.makedirs(path)
  for s in slices_list:
    int_slice = eval(s)[0]
    slice_img = load(mouse=mouse_name, folder=folder_data, slice_num=int_slice, is_png=is_png, to_resize=to_resize)
    temp_data = mouse_data.loc[mouse_data['Slices'] == s]
    cluster_list = list(dict.fromkeys(temp_data['Clusters'].tolist()))
    for c in cluster_list:
      colour = d[c]
      temp_data_2 = temp_data.loc[temp_data['Clusters'] == c]
      patches_list = list(dict.fromkeys(temp_data_2['Patch'].tolist()))
      for p in patches_list:
        int_patch = eval(p)
        mark_patch(slice_img, int_patch, colour)
    im = Image.fromarray(slice_img)
    im.save(path + "/" + mouse_name + "_" + str(int_slice) + "_" + network + ".png")

The main function - the automatic creation of the marked slices

In [ ]:
'''
MICE_IDS is the names of the mice scans that their patches will be marked according to their cluster (and will be saved). If you want different scans,
just change this list.
file_name is the full path to the csv file, containing the data about the clusters.
network is the name of the folder that will contain the slices with marked patches, for you to choose
is_png - is the format of the data in PNG, for you it is probablly False
to_resize - in case the format is DICOM, true iff you need to resize the slices - mainly depends on whether you resized the slices when you created
the patch dataset (if you resized then, resize again. We decided to resize)
'''

MICE_IDS = ["MB1333220714F000H00000000C000000", "MB1444220714F000H00000000C000000", "MB1589241214FC00H00004052C000000",
            "MB1752210615FC00H00000557C000000", "MB1858230615FC00H00001912C000000", "MB1858140715FC00H00001912C000000",
            "MB1583050215FC00H00004052C000000", "MB1598010315FC00H00004457C000000", "MB1695040215F000H00000000C000000",
            "MB1588050215FC00H00004052C000000", "MB1472110315FC00H00004052C000000", "MB1755210615FC00H00000557C000000",
            "MB1953010315FC00H00004457C000000", "MB1468010315FC00H00003912C000000", "MB1513090614F000H00000000C000000",
            "MB1363150215F000H00000000C000000", "MB1370270414F000H00000000C000000", "MB1408220714F000H00000000C000000",
             "MB1452010315FC00H00000188C000000", "MB1512290714F000H00000000C000000",
             "MB1539110315FC00H00001912C000000", "MB1747010315FC00H00000072C000000", "MB1317200714F000H00000000C000000",
            "MB1464210615FC00H00000519C000000", "MB1488290614F000H00000000C000000", "MB1701240614F000H00000000C000000",
            "MB1727080914F000H00000000C000000", "MB1790241214FC00H00003348C000000"]

file_name = 'results/clustering/segmentation/eps_0.38_minsamples_100/clusters.csv'
data = get_data(file_name=file_name)
for mouse in MICE_IDS:
  mark_patches(data, mouse, folder_data="datasets/miceData", network="eps_0.38_minsamples_100", is_png=False, to_resize=True)

'''
Examples of working with PNG saved files:

file_name = 'results/clustering/segmentation/eps_0.38_minsamples_100/clusters.csv'
data = get_data(file_name=file_name)
for mouse in MICE_IDS:
  mark_patches(data, mouse, network="seg_eps_0.38_minsamples_100")

file_name = 'results/features/classification_2d_features_from_model_12_epochs.csv_clusters.csv'
data = get_data(file_name=file_name)
for mouse in MICE_IDS:
  mark_patches(data, mouse, network="classification")

file_name = "results/features/segmentation_features_from_model_67_epochs.csv_clusters.csv"
data = get_data(file_name=file_name)
for mouse in MICE_IDS:
  mark_patches(data, mouse, network="segmentation")
'''

Marks of specific patches - probably you won't need it

In [ ]:
mouse_name  = "MB1755210615FC00H00000557C000000"
slice_num = 317
slice_cl = load(mouse_name, slice_num, folder="datasets/newMiceData")
slice_seg = load(mouse_name, slice_num, folder="datasets/newMiceData")
slice_seg_3_clusters = load(mouse_name, slice_num, folder="datasets/newMiceData")

mark_patch(slice_cl, (1,3), "red")
mark_patch(slice_seg, (1,3), "red")
mark_patch(slice_seg_3_clusters, (1,3), "blue")
mark_patch(slice_cl, (2,1), "blue")
mark_patch(slice_seg, (2,1), "blue")
mark_patch(slice_seg_3_clusters, (2,1), "yellow")
mark_patch(slice_cl, (3,2), "blue")
mark_patch(slice_seg, (3,2), "red")
mark_patch(slice_seg_3_clusters, (3,2), "red")

im = Image.fromarray(slice_cl)
im.save("MB1755210615FC00H00000557C000000_317_cl" + ".png")
im = Image.fromarray(slice_seg)
im.save("MB1755210615FC00H00000557C000000_317_seg" + ".png")
im = Image.fromarray(slice_seg_3_clusters)
im.save("MB1755210615FC00H00000557C000000_317_seg_3_clusters" + ".png")
